In [ ]:
!pip install torch
!pip install torch_geometric
!pip install ax_platform
!pip install botorch

In [ ]:
import copy
import os.path as osp

import torch
import torch.nn.functional as F
from torch.nn import GRU, Linear, ReLU, Sequential

import torch_geometric.transforms as T
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader
from torch_geometric.nn import NNConv, Set2Set
from torch_geometric.utils import remove_self_loops

target = 0
dim = 64


class MyTransform:
    def __call__(self, data):
        data = copy.copy(data)
        data.y = data.y[:, target]  # Specify target.
        return data


class Complete:
    def __call__(self, data):
        data = copy.copy(data)
        device = data.edge_index.device

        row = torch.arange(data.num_nodes, dtype=torch.long, device=device)
        col = torch.arange(data.num_nodes, dtype=torch.long, device=device)

        row = row.view(-1, 1).repeat(1, data.num_nodes).view(-1)
        col = col.repeat(data.num_nodes)
        edge_index = torch.stack([row, col], dim=0)

        edge_attr = None
        if data.edge_attr is not None:
            idx = data.edge_index[0] * data.num_nodes + data.edge_index[1]
            size = list(data.edge_attr.size())
            size[0] = data.num_nodes * data.num_nodes
            edge_attr = data.edge_attr.new_zeros(size)
            edge_attr[idx] = data.edge_attr

        edge_index, edge_attr = remove_self_loops(edge_index, edge_attr)
        data.edge_attr = edge_attr
        data.edge_index = edge_index

        return data


path = osp.join(osp.dirname(osp.realpath("__file__")), '..', 'data', 'QM9')
transform = T.Compose([MyTransform(), Complete(), T.Distance(norm=False)])
# Originalmente: Carga y preparación del dataset
dataset = QM9(path, transform=transform).shuffle()

# Modificación para usar un subconjunto más pequeño
# Por ejemplo, usar solo los primeros 10,000 datos del dataset
subset_size = 1000  # Define el tamaño del subconjunto
subset_indices = torch.randperm(len(dataset))[:subset_size]  # Selecciona índices al azar
dataset = dataset[subset_indices]

# Procede como antes
mean = dataset.data.y.mean(dim=0, keepdim=True)
std = dataset.data.y.std(dim=0, keepdim=True)
dataset.data.y = (dataset.data.y - mean) / std
mean, std = mean[:, target].item(), std[:, target].item()

# Split datasets teniendo en cuenta el nuevo tamaño
test_dataset = dataset[:int(subset_size*0.2)]
val_dataset = dataset[int(subset_size*0.2):int(subset_size*0.4)]
train_dataset = dataset[int(subset_size*0.4):]
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

#________________________________________________________________________________________________
#RED NEURONAL ARQUITECTURA

class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lin0 = torch.nn.Linear(dataset.num_features, dim)

        nn = Sequential(Linear(5, 128), ReLU(), Linear(128, dim * dim))
        self.conv = NNConv(dim, dim, nn, aggr='mean')
        self.gru = GRU(dim, dim)

        self.set2set = Set2Set(dim, processing_steps=3)
        self.lin1 = torch.nn.Linear(2 * dim, dim)
        self.lin2 = torch.nn.Linear(dim, 1)
#________________________________________________________________________________________________
#PROPAGACIÓN DE INFORMACIÓN
    def forward(self, data):
        out = F.relu(self.lin0(data.x))
        h = out.unsqueeze(0)

        for i in range(3):
            m = F.relu(self.conv(out, data.edge_index, data.edge_attr))
            out, h = self.gru(m.unsqueeze(0), h)
            out = out.squeeze(0)

        out = self.set2set(out, data.batch)
        out = F.relu(self.lin1(out))
        out = self.lin2(out)
        return out.view(-1)
#________________________________________________________________________________________________

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                       factor=0.7, patience=5,
                                                       min_lr=0.00001) #ajusta el LR durante el entrenamiento.
#________________________________________________________________________________________________
#ENTRENAMIENTO DE RED NEURONAL

def train(epoch):
    model.train()
    loss_all = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        loss = F.mse_loss(model(data), data.y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
    return loss_all / len(train_loader.dataset)

#_______________________________________________________________________________________________
#TEST RED NEURONAL
def test(loader):
    model.eval()
    error = 0

    for data in loader:
        data = data.to(device)
        error += (model(data) * std - data.y * std).abs().sum().item()  # MAE
    return error / len(loader.dataset)
#_______________________________________________________________________________________________
#ENTRENAMIENTO DEL MODELO EN EPOCAS
best_val_error = None
for epoch in range(1, 21):
    lr = scheduler.optimizer.param_groups[0]['lr']
    loss = train(epoch)
    val_error = test(val_loader)
    scheduler.step(val_error)

    if best_val_error is None or val_error <= best_val_error:
        test_error = test(test_loader)
        best_val_error = val_error

    print(f'Epoch: {epoch:03d}, LR: {lr:7f}, Loss: {loss:.7f}, '
          f'Val MAE: {val_error:.7f}, Test MAE: {test_error:.7f}')

In [ ]:
import copy
import os.path as osp

import torch
import torch.nn.functional as F
from torch.nn import GRU, Linear, ReLU, Sequential

import torch_geometric.transforms as T
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader
from torch_geometric.nn import NNConv, Set2Set
from torch_geometric.utils import remove_self_loops

from botorch.models import SingleTaskGP
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf
from botorch.utils import standardize

target = 0
dim = 64


class MyTransform:
    def __call__(self, data):
        data = copy.copy(data)
        data.y = data.y[:, target]  # Specify target.
        return data


class Complete:
    def __call__(self, data):
        data = copy.copy(data)
        device = data.edge_index.device

        row = torch.arange(data.num_nodes, dtype=torch.long, device=device)
        col = torch.arange(data.num_nodes, dtype=torch.long, device=device)

        row = row.view(-1, 1).repeat(1, data.num_nodes).view(-1)
        col = col.repeat(data.num_nodes)
        edge_index = torch.stack([row, col], dim=0)

        edge_attr = None
        if data.edge_attr is not None:
            idx = data.edge_index[0] * data.num_nodes + data.edge_index[1]
            size = list(data.edge_attr.size())
            size[0] = data.num_nodes * data.num_nodes
            edge_attr = data.edge_attr.new_zeros(size)
            edge_attr[idx] = data.edge_attr

        edge_index, edge_attr = remove_self_loops(edge_index, edge_attr)
        data.edge_attr = edge_attr
        data.edge_index = edge_index

        return data


path = osp.join(osp.dirname(osp.realpath("__file__")), '..', 'data', 'QM9')
transform = T.Compose([MyTransform(), Complete(), T.Distance(norm=False)])
# Originalmente: Carga y preparación del dataset
dataset = QM9(path, transform=transform).shuffle()

# Modificación para usar un subconjunto más pequeño
# Por ejemplo, usar solo los primeros 10,000 datos del dataset
subset_size = 1000  # Define el tamaño del subconjunto
subset_indices = torch.randperm(len(dataset))[:subset_size]  # Selecciona índices al azar
dataset = dataset[subset_indices]

# Procede como antes
mean = dataset.data.y.mean(dim=0, keepdim=True)
std = dataset.data.y.std(dim=0, keepdim=True)
dataset.data.y = (dataset.data.y - mean) / std
mean, std = mean[:, target].item(), std[:, target].item()

# Split datasets teniendo en cuenta el nuevo tamaño
test_dataset = dataset[:int(subset_size*0.2)]
val_dataset = dataset[int(subset_size*0.2):int(subset_size*0.4)]
train_dataset = dataset[int(subset_size*0.4):]
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

#________________________________________________________________________________________________
#RED NEURONAL ARQUITECTURA


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.num_neurons = num_neurons

        self.lin0 = torch.nn.Linear(dataset.num_features, dim)

        nn = Sequential(Linear(5, 128), ReLU(), Linear(self.num_neurons, dim * dim))
        self.conv = NNConv(dim, dim, nn, aggr='mean')
        self.gru = GRU(dim, dim)

        self.set2set = Set2Set(dim, processing_steps=3)
        self.lin1 = torch.nn.Linear(2 * dim, dim)
        self.lin2 = torch.nn.Linear(dim, 1)

#________________________________________________________________________________________________
#PROPAGACIÓN DE INFORMACIÓN
    def forward(self, data):
        out = F.relu(self.lin0(data.x))
        h = out.unsqueeze(0)

        for i in range(3):
            m = F.relu(self.conv(out, data.edge_index, data.edge_attr))
            out, h = self.gru(m.unsqueeze(0), h)
            out = out.squeeze(0)

        out = self.set2set(out, data.batch)
        out = F.relu(self.lin1(out))
        out = self.lin2(out)
        return out.view(-1)

#________________________________________________________________________________________________
# Define the hyperparameter optimization settings
num_neurons_range = [64, 128, 256, 512]
num_epochs = 20
num_iterations = 10

# Define the objective function for hyperparameter optimization
def objective(num_neurons):
    val_errors = []
    for _ in range(num_iterations):
        model = Net(num_neurons=num_neurons)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=5, min_lr=0.00001)
        best_val_error = None
        for epoch in range(1, 21):
            model.train()
            loss_all = 0
            for data in train_loader:
                data = data.to(device)
                optimizer.zero_grad()
                loss = F.mse_loss(model(data), data.y)
                loss.backward()
                loss_all += loss.item() * data.num_graphs
                optimizer.step()
            loss = loss_all / len(train_loader.dataset)
            model.eval()
            error = 0
            for data in val_loader:
                data = data.to(device)
                error += (model(data) * std - data.y * std).abs().sum().item()  # MAE
            val_error = error / len(val_loader.dataset)
            scheduler.step(val_error)
            if best_val_error is None or val_error <= best_val_error:
                test_error = 0
                for data in test_loader:
                    data = data.to(device)
                    error += (model(data) * std - data.y * std).abs().sum().item()  # MAE
                best_val_error = val_error
        val_errors.append(best_val_error)
    return sum(val_errors) / len(val_errors)

#________________________________________________________________________________________________

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)

dtype = torch.double
bounds = torch.tensor([[64.0], [512.0]], device=device, dtype=dtype)
search_space = torch.tensor([[64.0], [128.0], [256.0], [512.0]], device=device, dtype=dtype)
val_dataset = torch.tensor([objective(lr) for lr in search_space], device=device, dtype=dtype)
gp = SingleTaskGP(search_space, val_dataset)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)

# Perform Bayesian Optimization
best_acqf_value = torch.tensor(float("inf"))
best_candidate = None
for _ in range(num_candidates):
    # Optimize acquisition function
    candidate, acqf_value = optimize_acqf(
        acq_function=acq_function,
        bounds=bounds,
        q=1,
        num_restarts=10,
        raw_samples=512,
    )
    # Update current best candidate
    if acqf_value < best_acqf_value:
        best_acqf_value = acqf_value
        best_candidate = candidate
